<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/ComfyUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🚀 ComfyUI Ultimate Launcher (Protocol Fix)
# @markdown ### ℹ️ Project Info
# @markdown This script launches **ComfyUI** with a self-healing Cloudflare tunnel.
# @markdown **Patch Applied:** Forced HTTP2 protocol to fix "Invalid UUID" errors.

# @markdown ---
# @markdown ### ⚙️ Storage Settings
USE_GOOGLE_DRIVE = False # @param {type:"boolean"}
# @markdown <font size="2" color="gray">✅ **Checked**: Saves to Google Drive.<br>❌ **Unchecked**: Temporary Colab storage.</font>

# @markdown ---
# @markdown ### 🔄 Update Settings
UPDATE_COMFY_UI = True # @param {type:"boolean"}

import os
import subprocess
import time
import urllib.request
import re
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path

# --- 1. UI SETUP ---
header = widgets.HTML("<h2>🎨 ComfyUI Stability Launcher</h2>")

info_html = widgets.HTML("""
<div style="padding: 10px; background-color: #212121; color: white; border-left: 4px solid #4caf50; font-family: sans-serif; border-radius: 4px;">
    <b>Status:</b> Initializing...<br>
    <b>Note:</b> If Cloudflare fails, the script will auto-restart the tunnel.
</div>
""")

status_label = widgets.Label(value="Initializing...", style={'description_width': 'initial'})
progress_bar = widgets.IntProgress(value=0, min=0, max=100, layout=widgets.Layout(width='100%'))
link_area = widgets.Output()
log_area = widgets.Textarea(value="", placeholder="Logs...", layout=widgets.Layout(width='100%', height='250px'))
log_acc = widgets.Accordion(children=[log_area])
log_acc.set_title(0, '📝 System Logs (Click to expand)')

display(widgets.VBox([header, info_html, status_label, progress_bar, link_area, log_acc]))

# --- 2. FUNCTIONS ---
ROOT = Path("/content")

def log(msg, important=False):
    timestamp = time.strftime("%H:%M:%S")
    log_area.value += f"[{timestamp}] {msg}\n"
    if important:
        status_label.value = f"👉 {msg}"

def kill_zombies():
    log("🧹 Killing old processes...")
    subprocess.run(["pkill", "-f", "cloudflared"], check=False)
    subprocess.run(["pkill", "-f", "comfyui"], check=False)
    subprocess.run(["fuser", "-k", "8188/tcp"], capture_output=True, check=False)

def run_cmd(cmd, cwd=None):
    process = subprocess.Popen(cmd, cwd=cwd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None: break
        if output: log(output.strip())
    if process.returncode != 0: raise Exception(f"Command failed: {cmd}")

# --- 3. MAIN EXECUTION ---
try:
    kill_zombies()

    # A. Storage
    if USE_GOOGLE_DRIVE:
        from google.colab import drive
        if not os.path.exists('/content/drive'): drive.mount('/content/drive')
        INSTALL_DIR = Path("/content/drive/MyDrive/ComfyUI")
    else:
        INSTALL_DIR = Path("/content/ComfyUI")

    progress_bar.value = 10

    # B. Install
    if not INSTALL_DIR.exists():
        log("⬇️ Cloning ComfyUI...", important=True)
        run_cmd(f"git clone https://github.com/comfyanonymous/ComfyUI {INSTALL_DIR}")
    elif UPDATE_COMFY_UI:
        log("🔄 Updating ComfyUI...", important=True)
        run_cmd("git pull", cwd=INSTALL_DIR)

    progress_bar.value = 30

    # C. Manager
    MANAGER_DIR = INSTALL_DIR / "custom_nodes" / "ComfyUI-Manager"
    if not MANAGER_DIR.exists():
        log("📦 Installing Manager...", important=True)
        run_cmd(f"git clone https://github.com/ltdrdata/ComfyUI-Manager {MANAGER_DIR}")

    progress_bar.value = 50

    # D. Dependencies
    log("🛠️ Installing Dependencies...", important=True)
    run_cmd(f"pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121", cwd=INSTALL_DIR)

    progress_bar.value = 70

    # E. Cloudflare (Force Redownload)
    CLOUDFLARED = ROOT / "cloudflared-linux-amd64"
    if CLOUDFLARED.exists(): CLOUDFLARED.unlink()

    log("☁️ Downloading Cloudflare Tunnel...", important=True)
    run_cmd(f"wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O {CLOUDFLARED}")
    os.chmod(CLOUDFLARED, 0o777)

    progress_bar.value = 85

    # F. Launch Services
    log("⚡ Starting Services...", important=True)
    tunnel_log = ROOT / "cloudflared.log"
    if tunnel_log.exists(): tunnel_log.unlink()

    # Start Tunnel (WITH HTTP2 FORCE FIX)
    # We add --protocol http2 to prevent the UUID error
    tunnel_cmd = f"{CLOUDFLARED} tunnel --protocol http2 --url http://127.0.0.1:8188 --metrics localhost:45678 > {tunnel_log} 2>&1 &"
    subprocess.Popen(tunnel_cmd, shell=True)

    # Start ComfyUI
    comfy_cmd = f"python main.py --dont-print-server --listen --port 8188"
    comfy_proc = subprocess.Popen(comfy_cmd, cwd=INSTALL_DIR, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

    progress_bar.value = 95
    progress_bar.bar_style = 'warning'

    # G. Monitor
    status_label.value = "⏳ Waiting for URL..."
    url = None
    is_ready = False
    tunnel_retries = 0

    while True:
        # 1. Check ComfyUI
        if comfy_proc.poll() is not None:
            log("❌ ComfyUI Crashed!", important=True)
            progress_bar.bar_style = 'danger'
            break

        # 2. Check Localhost
        if not is_ready:
            try:
                with urllib.request.urlopen("http://127.0.0.1:8188", timeout=1) as response:
                    if response.getcode() == 200:
                        is_ready = True
                        log("✅ ComfyUI is listening on localhost")
            except:
                pass

        # 3. Check Tunnel
        if url is None and tunnel_log.exists():
            try:
                with open(tunnel_log, "r") as f:
                    content = f.read()
                    found = re.search(r'https://[\w-]+\.trycloudflare\.com', content)
                    if found:
                        url = found.group(0)
                        log(f"🔗 Tunnel URL: {url}")

                    # ERROR DETECTION & AUTO-RESTART
                    if "failed to parse quick Tunnel ID" in content and tunnel_retries < 3:
                        log("⚠️ Cloudflare Error detected. Restarting tunnel...", important=True)
                        subprocess.run(["pkill", "-f", "cloudflared"], check=False)
                        if tunnel_log.exists(): tunnel_log.unlink()
                        time.sleep(2)
                        # Restart with HTTP2 forced again
                        subprocess.Popen(tunnel_cmd, shell=True)
                        tunnel_retries += 1
                        log(f"🔄 Tunnel Restart Attempt {tunnel_retries}/3")

            except:
                pass

        # 4. Success
        if is_ready and url:
            progress_bar.value = 100
            progress_bar.bar_style = 'success'
            status_label.value = "✅ System Operational"
            with link_area:
                link_area.clear_output()
                display(widgets.HTML(f"""
                <div style="text-align: center; margin: 20px;">
                    <a href="{url}" target="_blank" style="background-color: #28a745; color: white; padding: 15px 30px; font-size: 18px; font-weight: bold; text-decoration: none; border-radius: 8px;">🚀 OPEN COMFYUI</a>
                </div>
                """))

            # Stream logs
            line = comfy_proc.stdout.readline()
            if line: log(line.strip())

        time.sleep(0.5)

except KeyboardInterrupt:
    log("🛑 Stopped.")
    kill_zombies()
except Exception as e:
    log(f"❌ Error: {e}", important=True)